# Notebook Plan

#### Create Analysis File to run Regressions

Use merged and cleaned data to create standardized regression ready files

1. Read in Data
2. Feature Engineering
3. Create Percent Change DataFrame

In [47]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt
import seaborn as sns

from IPython.display import display

%matplotlib inline 

## 1. Read in Data

In [48]:
init_df = pd.read_csv('data/final_analysis_file.csv')

In [49]:
init_df.head()

,Unnamed: 0,Award Mean,Award Sum,Company Count,Score1,Score2,Score5,Score6,Score7,Unnamed: 0.1,...,recipient_amount,recipient_count,regular_employees,regular_establishments,total_earned_bachelor,total_earned_graduate_degree,total_foreign,total_less_than_bachelor,total_native,total_population
0,0,128725.714286,901080.0,7.0,2.351350,2.675760,2.328623,2.718282,5.046905,0,...,1.741934e+08,873.0,157590.0,6512.0,19771.0,11970.0,35694.0,50236.0,66410.0,102361.0
1,1,0.000000,0.0,0.0,1.713367,2.368286,1.425096,2.691711,4.116807,1,...,0.000000e+00,0.0,NaN,NaN,675.0,579.0,441.0,1169.0,3089.0,3461.0
2,2,166188.029412,5650393.0,34.0,2.571461,2.718282,2.348135,2.672531,5.020666,2,...,2.363753e+09,3752.0,2895473.0,219720.0,222953.0,117707.0,516105.0,1079063.0,1438743.0,1953631.0
3,3,238464.416667,5723146.0,24.0,2.718282,2.702366,2.718282,2.546716,5.264998,3,...,3.377225e+08,849.0,427571.0,34442.0,125900.0,61878.0,329757.0,470458.0,564132.0,894943.0
4,4,129946.000000,129946.0,1.0,2.240925,2.344083,2.410876,2.464089,4.874966,4,...,2.006920e+08,690.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2. Feature Engineering

This section involves creating our scoring method.
    1. Score_invented: Score1 + Score5 (Invented Patents + Invented Patent Citations)
    2. Score_assigned: Score2 + Score6 (Assigned Patents + Assigned Patent Citations)
    
Additionally, this section includes creating any new features (percentages/scaled percentages) and any standardizing

In [50]:
init_df['Score_invented'] = init_df['Score1'] + init_df['Score5']
init_df['Score_assigned'] = init_df['Score2'] + init_df['Score6']
init_df.drop(['Score1', 'Score2', 'Score5', 'Score6', 'Score7', 'Unnamed: 0', 'Unnamed: 0.1'], axis = 1, inplace = True)

In [51]:
init_df.shape

(13973, 23)

In [52]:
init_df.dropna(inplace = True)
init_df.shape

(8274, 23)

In [53]:
for year in range(2001, 2015):
    temp = init_df.loc[init_df.Year == year]
    print(temp.shape)

(558, 23)
(680, 23)
(689, 23)
(713, 23)
(713, 23)
(686, 23)
(699, 23)
(694, 23)
(695, 23)
(699, 23)
(704, 23)
(744, 23)
(0, 23)
(0, 23)


In [54]:
init_df.columns

Index(['Award Mean', 'Award Sum', 'Company Count', 'Year', 'city_state',
       'creative_employees', 'creative_establishments', 'empowerment_zone',
       'median_household_income', 'performance_amount', 'performance_count',
       'recipient_amount', 'recipient_count', 'regular_employees',
       'regular_establishments', 'total_earned_bachelor',
       'total_earned_graduate_degree', 'total_foreign',
       'total_less_than_bachelor', 'total_native', 'total_population',
       'Score_invented', 'Score_assigned'],
      dtype='object')

In [57]:
#Processing function to create new features, standardizes values and returns a regression ready dataframe

def processing(init_df):
    ## Create all scaled percentages and ratios

    init_df['percent_creative_class'] = init_df['creative_employees'] / (init_df['creative_employees'] + init_df['regular_employees'])
    init_df['scaled_perc_creative_class'] = (init_df['percent_creative_class'] * np.log(init_df['creative_employees'] + init_df['regular_employees'])) / 10

    init_df['creative_establishment_ratio'] = init_df['creative_establishments'] / (init_df['regular_establishments'] + init_df['creative_establishments'])
    init_df['company_count_perc'] = init_df['Company Count'] / (init_df['regular_establishments'] + init_df['creative_establishments'])

    init_df['percent_bachelors'] = init_df['total_earned_bachelor'] / (init_df['total_earned_bachelor'] + init_df['total_earned_graduate_degree'] + init_df['total_less_than_bachelor'])
    init_df['percent_graduate'] = init_df['total_earned_graduate_degree'] / (init_df['total_earned_bachelor'] + init_df['total_earned_graduate_degree'] + init_df['total_less_than_bachelor'])

    init_df['percent_foreign_born'] = init_df['total_foreign'] / init_df['total_population']

    init_df['scaled_population'] = np.log(init_df['total_population'])


    init_df['recipient_mean'] = init_df['recipient_amount'] / init_df['recipient_count']
    init_df['performance_mean'] = init_df['performance_amount'] / init_df['performance_count']
    init_df = init_df.fillna(0)


    display(init_df.isna().sum())
    
    
    #write to file. This will be used for visualization
    init_df.to_csv("pre_scaling_data.csv")
    
    ## Standardize all non-binary and non-text features
    init_list = []
    years = list(init_df.Year.unique())
    for year in years:
        temp = init_df.loc[init_df.Year == year].reset_index()
        temp['Award Mean'] = temp['Award Mean']**(1/np.log(temp['Award Mean'].max()))
        temp['recipient_mean'] = temp.recipient_mean**(1/np.log(temp.recipient_mean.max()))
        temp['performance_mean'] = temp.performance_mean**(1/np.log(temp.performance_mean.max()))
        temp = temp.fillna(0) #NaNs occur at performance and recipient means => just 0 values.
        cities = temp['city_state']
        empowerment_zone = temp['empowerment_zone']
        temp = temp.drop(['city_state', 'empowerment_zone', 'index'], axis = 1)
        columns = temp.columns


        scaler = StandardScaler()
        print(scaler.fit(temp))
        temp_scaled = scaler.transform(temp)
        temp_scaled = pd.DataFrame(scaler.fit_transform(temp_scaled))

        temp_scaled.columns = columns
        temp_scaled['city_state'] = cities
        temp_scaled['empowerment_zone'] = empowerment_zone
        temp_scaled['Year'] = year

        init_list.append(temp_scaled)
        del temp, temp_scaled


    scaled_df = pd.concat(init_list, ignore_index = True)
    scaled_df.rename(columns={'Award Mean': 'award_mean', 'Award Sum': 'award_sum', 'Company Count': 'company_count'},
                inplace = True)

    display(scaled_df.head())
    
    return(scaled_df)
    

In [56]:
trial_df = processing(init_df)
trial_df.to_csv('data/regression_ready_df.csv')

Award Mean                      0
Award Sum                       0
Company Count                   0
Year                            0
city_state                      0
creative_employees              0
creative_establishments         0
empowerment_zone                0
median_household_income         0
performance_amount              0
performance_count               0
recipient_amount                0
recipient_count                 0
regular_employees               0
regular_establishments          0
total_earned_bachelor           0
total_earned_graduate_degree    0
total_foreign                   0
total_less_than_bachelor        0
total_native                    0
total_population                0
Score_invented                  0
Score_assigned                  0
percent_creative_class          0
scaled_perc_creative_class      0
creative_establishment_ratio    0
company_count_perc              0
percent_bachelors               0
percent_graduate                0
percent_foreig

StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)
StandardScaler(copy=True, with_mean=True, with_std=True)


,award_mean,award_sum,company_count,Year,creative_employees,creative_establishments,median_household_income,performance_amount,performance_count,recipient_amount,...,creative_establishment_ratio,company_count_perc,percent_bachelors,percent_graduate,percent_foreign_born,scaled_population,recipient_mean,performance_mean,city_state,empowerment_zone
0,0.866729,-0.135717,-0.005774,2001,-0.020634,-0.170013,1.152891,-0.020528,-0.118605,-0.012129,...,-0.691198,-0.093655,0.737127,0.706716,1.935172,0.362860,0.919007,1.885741,santaclara_ca,0
1,0.904298,1.062714,1.444016,2001,2.954312,7.382901,-0.636831,6.510660,1.917271,3.795957,...,0.518296,-0.388456,-0.372824,-0.333978,1.166876,2.976969,1.066948,1.843079,houston_tx,1
2,0.958632,1.081072,0.907057,2001,0.536590,0.867351,1.195224,-0.153219,0.004782,0.272281,...,0.428170,-0.218982,0.078728,-0.150975,2.114772,2.284925,1.006517,1.254293,sanjose_ca,0
3,0.967675,1.552161,1.229232,2001,-0.076239,-0.177978,1.422194,0.854559,-0.010642,1.992505,...,-0.637564,1.188322,1.225772,1.660487,2.352124,0.586670,1.179324,1.955497,sunnyvale_ca,0
4,0.912048,0.653006,0.853361,2001,21.154801,16.914971,-0.545465,-0.118428,1.747615,1.147251,...,-0.273663,-0.426346,-0.417356,-0.013156,2.024254,4.227571,0.924553,1.243803,newyork_ny,1


## 3. Get percent change for each city

Get a dataframe of percentage changes for each feature to identify any other trends

In [2]:
perc_diff_df = pd.read_csv('patent_data_all_years.csv')

In [3]:
# Only get cities that occur in every year we are looking at.

perc_diff_df = perc_diff_df.loc[(perc_diff_df.Year > 2000) & (perc_diff_df.Year < 2013)]
city_vals = pd.DataFrame(perc_diff_df.city_state.value_counts()).reset_index()
city_vals = city_vals.loc[city_vals.city_state > 11]

In [61]:
city_vals = list(city_vals['index'])

In [62]:
def is_city(x):
    if x in city_vals:
        return(1)
    else:
        return(0)

In [63]:
perc_diff_df['cities_to_keep'] = perc_diff_df.city_state.apply(lambda x: is_city(x))

In [64]:
perc_diff_df = perc_diff_df.loc[perc_diff_df.cities_to_keep == 1]

In [65]:
perc_diff_df.isna().sum()

Unnamed: 0                              0
Unnamed: 0.1                            0
Unnamed: 0.1.1                          0
City                                   12
Inv_to_Assignee_ratio                   0
Patents                                 0
State                                  60
Year                                    0
assignee_IPC_A                          0
assignee_IPC_B                          0
assignee_IPC_C                          0
assignee_IPC_D                          0
assignee_IPC_E                          0
assignee_IPC_F                          0
assignee_IPC_G                          0
assignee_IPC_H                          0
assignee_pats_cited                     0
assignee_pats_cited_ratio               0
assignee_type_2                         0
assignee_type_3                         0
assignee_type_4                         0
assignee_type_5                         0
assignee_type_6                         0
assignee_type_7                   

In [66]:
perc_diff_df.drop(['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'], axis = 1, inplace = True)
perc_diff_df = perc_diff_df[['City', 'State', 'Year', 'Patents', 'assignee_pats_cited',
                             'inventor_patents', 'inventor_pats_cited']]
perc_diff_df = perc_diff_df[pd.notnull(perc_diff_df['City'])]
perc_diff_df = perc_diff_df[pd.notnull(perc_diff_df['State'])]
perc_diff_df['City'] = perc_diff_df.City.apply(lambda x: str.lower(x))
perc_diff_df['State'] = perc_diff_df.State.apply(lambda x: str.lower(x))
perc_diff_df['city_state'] = perc_diff_df['City'] + '_' + perc_diff_df['State']
perc_diff_df.drop(['City', 'State'], axis = 1, inplace = True)

In [69]:
perc_diff_df.head(2)
cities = list(perc_diff_df.city_state.unique())
display(cities[:5])

['santa clara_ca',
 'armonk_ny',
 'houston_tx',
 'san jose_ca',
 'boise_id',
 'sunnyvale_ca',
 'new york_ny',
 'wilmington_de',
 'palo alto_ca',
 'san diego_ca',
 'dallas_tx',
 'washington_dc',
 'redmond_wa',
 'rochester_ny',
 'st. paul_mn',
 'troy_mi',
 'austin_tx',
 'irvine_ca',
 'schenectady_ny',
 'milpitas_ca',
 'chicago_il',
 'stamford_ct',
 'cambridge_ma',
 'cincinnati_oh',
 'schaumburg_il',
 'minneapolis_mn',
 'dearborn_mi',
 'oakland_ca',
 'fremont_ca',
 'mountain view_ca',
 'seattle_wa',
 'scotts valley_ca',
 'philadelphia_pa',
 'boston_ma',
 'morristown_nj',
 'murray hill_nj',
 'neenah_wi',
 'cleveland_oh',
 'st. louis_mo',
 'indianapolis_in',
 'staten island_ny',
 'princeton_nj',
 'maple grove_mn',
 'los angeles_ca',
 'atlanta_ga',
 'allentown_pa',
 'south san francisco_ca',
 'reno_nv',
 'pasadena_ca',
 'detroit_mi',
 'akron_oh',
 'auburn hills_mi',
 'durham_nc',
 'newark_de',
 'carlsbad_ca',
 'pittsburgh_pa',
 'el segundo_ca',
 'eden prairie_mn',
 'bethesda_md',
 'cupertino

In [78]:
trial_list = []

In [79]:
for i in cities:
    temp_df = perc_diff_df.loc[perc_diff_df.city_state == i]
    temp_df['Patent_change'] = temp_df['Patents'].pct_change()
    temp_df['assignee_pats_cited_change'] = temp_df['assignee_pats_cited'].pct_change()
    temp_df['inventor_patents_change'] = temp_df['inventor_patents'].pct_change()
    temp_df['inventor_pats_cited_change'] = temp_df['inventor_pats_cited'].pct_change()
    temp_df.drop(['Patents', 'assignee_pats_cited', 'inventor_patents', 'inventor_pats_cited'], 
                 axis = 1, inplace = True)
    trial_list.append(temp_df)

/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/opt/rh/anaconda/root/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [89]:
patent_change_df = pd.concat(trial_list, ignore_index = True)

In [90]:
patent_change_df.dropna(inplace = True)
patent_change_df.city_state = patent_change_df.city_state.apply(lambda x: x.replace(" ", ""))
patent_change_df.head()

,Year,city_state,Patent_change,assignee_pats_cited_change,inventor_patents_change,inventor_pats_cited_change
1,2002,santaclara_ca,0.128829,0.571347,0.298742,1.295406
2,2003,santaclara_ca,-0.053310,0.098698,-0.165254,0.349996
3,2004,santaclara_ca,-0.045741,0.315186,0.011603,0.172921
4,2005,santaclara_ca,-0.073924,0.035666,-0.034409,0.238063
5,2006,santaclara_ca,-0.039568,0.232292,0.172235,0.210823


In [88]:
analysis_df = pd.read_csv('data/final_analysis_file.csv')
display(analysis_df.head(2))
analysis_df.columns

,Unnamed: 0,Award Mean,Award Sum,Company Count,Score1,Score2,Score5,Score6,Score7,Unnamed: 0.1,...,recipient_amount,recipient_count,regular_employees,regular_establishments,total_earned_bachelor,total_earned_graduate_degree,total_foreign,total_less_than_bachelor,total_native,total_population
0,0,128725.714286,901080.0,7.0,2.351350,2.675760,2.328623,2.718282,5.046905,0,...,1.741934e+08,873.0,157590.0,6512.0,19771.0,11970.0,35694.0,50236.0,66410.0,102361.0
1,1,0.000000,0.0,0.0,1.713367,2.368286,1.425096,2.691711,4.116807,1,...,0.000000e+00,0.0,NaN,NaN,675.0,579.0,441.0,1169.0,3089.0,3461.0


Index(['Unnamed: 0', 'Award Mean', 'Award Sum', 'Company Count', 'Score1',
       'Score2', 'Score5', 'Score6', 'Score7', 'Unnamed: 0.1', 'Year',
       'city_state', 'creative_employees', 'creative_establishments',
       'empowerment_zone', 'median_household_income', 'performance_amount',
       'performance_count', 'recipient_amount', 'recipient_count',
       'regular_employees', 'regular_establishments', 'total_earned_bachelor',
       'total_earned_graduate_degree', 'total_foreign',
       'total_less_than_bachelor', 'total_native', 'total_population'],
      dtype='object')

In [108]:
final_analysis_df = pd.merge(patent_change_df, scaled_df,  how='inner', left_on=['city_state','Year'], 
                             right_on = ['city_state','Year'])

In [111]:
display(final_analysis_df.head())
final_analysis_df.columns

,Year,city_state,Patent_change,assignee_pats_cited_change,inventor_patents_change,inventor_pats_cited_change,award_mean,award_sum,company_count,creative_employees,...,scaled_perc_creative_class,creative_establishment_ratio,company_count_perc,percent_bachelors,percent_graduate,percent_foreign_born,scaled_population,recipient_mean,performance_mean,empowerment_zone
0,2002,santaclara_ca,0.128829,0.571347,0.298742,1.295406,0.951703,0.350680,0.343455,-0.037338,...,0.599358,0.353570,0.001675,0.675279,0.654651,2.013615,0.565963,0.969677,1.183850,0
1,2003,santaclara_ca,-0.053310,0.098698,-0.165254,0.349996,0.954618,0.771700,0.560586,-0.039466,...,0.600581,0.348776,0.099165,0.676436,0.642126,2.023683,0.545118,0.926089,0.929842,0
2,2004,santaclara_ca,-0.045741,0.315186,0.011603,0.172921,0.881448,0.703676,0.521533,-0.075409,...,0.576177,0.344699,0.080580,0.696795,0.665516,2.060905,0.506780,0.913232,0.731091,0
3,2005,santaclara_ca,-0.073924,0.035666,-0.034409,0.238063,0.819101,0.793182,0.900822,-0.088569,...,0.547065,0.347264,0.290276,0.639100,0.622693,2.036668,0.472978,0.937173,0.797090,0
4,2006,santaclara_ca,-0.039568,0.232292,0.172235,0.210823,0.830739,0.803632,1.061602,-0.056583,...,0.590247,0.343887,0.263703,0.652904,0.657707,2.006918,0.534657,0.979746,1.103890,0


Index(['Year', 'city_state', 'Patent_change', 'assignee_pats_cited_change',
       'inventor_patents_change', 'inventor_pats_cited_change', 'award_mean',
       'award_sum', 'company_count', 'creative_employees',
       'creative_establishments', 'median_household_income',
       'performance_amount', 'performance_count', 'recipient_amount',
       'recipient_count', 'regular_employees', 'regular_establishments',
       'total_earned_bachelor', 'total_earned_graduate_degree',
       'total_foreign', 'total_less_than_bachelor', 'total_native',
       'total_population', 'Score_invented', 'Score_assigned',
       'percent_creative_class', 'scaled_perc_creative_class',
       'creative_establishment_ratio', 'company_count_perc',
       'percent_bachelors', 'percent_graduate', 'percent_foreign_born',
       'scaled_population', 'recipient_mean', 'performance_mean',
       'empowerment_zone'],
      dtype='object')

In [112]:
final_analysis_df.drop(['Score_invented', 'Score_assigned'], axis = 1, inplace = True)

In [113]:
final_analysis_df.to_csv('perc_change_analysis_df.csv')